In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as constants
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure

import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:

N = 150 #150 by 150 grid

init_random = np.random.random((N,N))
lattice = np.zeros((N, N), dtype=np.int8)
lattice[init_random>=0.25] = 1
lattice[init_random<0.25] = 0

plt.imshow(lattice)
plt.show()

In [ ]:
def get_energy(lattice, U, mew):
    # applies the nearest neighbours summation
    kern = generate_binary_structure(2, 1) 
    kern[1][1] = False
    arr = 0.5*lattice * convolve(lattice, kern, mode='constant', cval=0)
    return -U*arr.sum() - mew*lattice.sum()

In [ ]:
z=4
crit_T = 150.8 #argon

In [ ]:
def get_U(crit_T):
    return 8*constants.Boltzmann*crit_T/z

In [ ]:
U = get_U(150.8)

In [ ]:
mew_eq_thy = -z*U
print(mew_eq_thy)

In [ ]:
mew_eq = -2.05*10**-20
shift = 0.2
mew_high, mew_low = mew_eq_thy*(1-shift), mew_eq_thy*(1+shift)
print(mew_eq, mew_high, mew_low)

In [ ]:
def get_J(U):
    J = U/4
    return J

In [ ]:
J = get_J(U)

In [ ]:
def get_h(mew):
    h = (z*U+mew)/2
    return h

In [ ]:
h_eq_thy = get_h(mew_eq_thy)

h_eq = get_h(mew_eq)
h_low = get_h(mew_low)
h_high = get_h(mew_high)

In [ ]:
def get_B(T):
    B = 1/(constants.Boltzmann*T)
    return B

# Base Metropolis Algorithm

In [ ]:
@njit("Tuple((f8[:], f8[:]))(i1[:,:], i8, f8, f8, f8, f8)", nopython=True, nogil=True)
def metropolis(density_arr, times, B, J, h, energy):
    density_arr = density_arr.copy()
    net_density = np.zeros(times-1, dtype=np.float64)
    net_energy = np.zeros(times-1, dtype=np.float64)

    def get_spin(denisty):
        return 2*(denisty+1)

    for t in range(times-1):
        x = np.random.randint(0, N)
        y = np.random.randint(0, N)
        density_i = density_arr[x, y]
        density_f = 1 - density_i
        spin_i = get_spin(density_i)
        spin_f = -spin_i


        E_i = 0
        E_f = 0
        
        if x > 0:
            E_i += -spin_i * density_arr[x-1, y] * J
            E_f += -spin_f * density_arr[x-1, y] * J
        if x < N-1:
            E_i += -spin_i * density_arr[x+1, y] * J
            E_f += -spin_f * density_arr[x+1, y] * J
        if y > 0:
            E_i += -spin_i * density_arr[x, y-1] * J
            E_f += -spin_f * density_arr[x, y-1] * J
        if y < N-1:
            E_i += -spin_i * density_arr[x, y+1] * J
            E_f += -spin_f * density_arr[x, y+1] * J

        E_i += -spin_i*h
        E_f += -spin_f*h

        dE = E_f - E_i
        if (dE > 0) and (np.random.random() < np.exp(-B * dE)):
            density_arr[x, y] = density_f
            energy += dE
        elif dE <= 0:
            density_arr[x, y] = density_f
            energy += dE

        net_density[t] = density_arr.sum()
        net_energy[t] = energy

    return net_density, net_energy

# Metropolis Algorithm With Visualization

In [ ]:
@njit("Tuple((f8[:], f8[:], i1[:, :, :]))(i1[:,:], i8, i8, f8, f8, f8, f8)", nopython=True, nogil=True)
def metropolis_plot(density_arr, times, snapshot_interval, B, J, h, energy):
    density_arr = density_arr.copy()
    net_density = np.zeros(times-1, dtype=np.float64)
    net_energy = np.zeros(times-1, dtype=np.float64)
    num_snapshots = times//snapshot_interval
    density_his = np.zeros((num_snapshots, N, N), dtype=np.int8)

    def get_spin(denisty):
        return 2*(denisty+1)

    for t in range(times-1):
        x = np.random.randint(0, N)
        y = np.random.randint(0, N)
        density_i = density_arr[x, y]
        density_f = 1 - density_i
        spin_i = get_spin(density_i)
        spin_f = -spin_i


        E_i = 0
        E_f = 0
        
        if x > 0:
            E_i += -spin_i * density_arr[x-1, y] * J
            E_f += -spin_f * density_arr[x-1, y] * J
        if x < N-1:
            E_i += -spin_i * density_arr[x+1, y] * J
            E_f += -spin_f * density_arr[x+1, y] * J
        if y > 0:
            E_i += -spin_i * density_arr[x, y-1] * J
            E_f += -spin_f * density_arr[x, y-1] * J
        if y < N-1:
            E_i += -spin_i * density_arr[x, y+1] * J
            E_f += -spin_f * density_arr[x, y+1] * J

        E_i += -spin_i*h
        E_f += -spin_f*h

        dE = E_f - E_i
        if (dE > 0) and (np.random.random() < np.exp(-B * dE)):
            density_arr[x, y] = density_f
            energy += dE
        elif dE <= 0:
            density_arr[x, y] = density_f
            energy += dE

        net_density[t] = density_arr.sum()
        net_energy[t] = energy
        if t % snapshot_interval == 0:
            density_his[t//snapshot_interval] = density_arr.copy()

    return net_density, net_energy, density_his

In [ ]:
B = get_B(100)

In [ ]:
net_density_eq, net_energy_eq, density_his_eq = metropolis_plot(lattice, 1000000, 10000, B, J, h_eq, get_energy(lattice, U, mew_eq))
plt.imshow(density_his_eq[-1])
print(net_density_eq[-1])

In [ ]:
net_density_low, net_energy_low, density_his_low = metropolis_plot(lattice, 1000000, 10000, B, J, h_low, get_energy(lattice, U, mew_low))
plt.imshow(density_his_low[-1])
print(net_density_low[-1])

In [ ]:
net_density_high, net_energy_high, density_his_high = metropolis_plot(lattice, 1000000, 10000, B, J, h_high, get_energy(lattice, U, mew_high))
plt.imshow(density_his_high[-1])
print(net_density_high[-1])

In [ ]:
mpl.rcParams['animation.embed_limit'] = 2**128

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.zeros((N, N)), cmap='gray', vmin=0, vmax=1)

def update(frame):
    im.set_data(density_his_eq[frame])
    return [im]

ani = FuncAnimation(fig, update, frames=len(density_his_eq), blit=True)
HTML(ani.to_jshtml())

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.zeros((N, N)), cmap='gray', vmin=0, vmax=1)

def update(frame):
    im.set_data(density_his_low[frame])
    return [im]

ani = FuncAnimation(fig, update, frames=len(density_his_low), blit=True)
HTML(ani.to_jshtml())

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.zeros((N, N)), cmap='gray', vmin=0, vmax=1)

def update(frame):
    im.set_data(density_his_high[frame])
    return [im]

ani_high = FuncAnimation(fig, update, frames=len(density_his_high), blit=True)
HTML(ani_high.to_jshtml())

# Mew / Density

In [ ]:
@njit("f8(i1[:,:], i8, f8, f8, f8)", nopython=True, nogil=True)
def metropolis_density(density_arr, times, B, J, h):
    density_arr = density_arr.copy()

    def get_spin(denisty):
        return 2*(denisty+1)

    for t in range(times-1):
        x = np.random.randint(0, N)
        y = np.random.randint(0, N)
        density_i = density_arr[x, y]
        density_f = 1 - density_i
        spin_i = get_spin(density_i)
        spin_f = -spin_i


        E_i = 0
        E_f = 0
        
        if x > 0:
            E_i += -spin_i * density_arr[x-1, y] * J
            E_f += -spin_f * density_arr[x-1, y] * J
        if x < N-1:
            E_i += -spin_i * density_arr[x+1, y] * J
            E_f += -spin_f * density_arr[x+1, y] * J
        if y > 0:
            E_i += -spin_i * density_arr[x, y-1] * J
            E_f += -spin_f * density_arr[x, y-1] * J
        if y < N-1:
            E_i += -spin_i * density_arr[x, y+1] * J
            E_f += -spin_f * density_arr[x, y+1] * J

        E_i += -spin_i*h
        E_f += -spin_f*h

        dE = E_f - E_i
        if (dE > 0) and (np.random.random() < np.exp(-B * dE)):
            density_arr[x, y] = density_f
        elif dE <= 0:
            density_arr[x, y] = density_f

    return  density_arr.sum()

In [ ]:
@njit("Tuple((f8[:], f8[:]))(i1[:,:], f8, f8, f8, f8, f8, f8, i8)", nopython=True, nogil=True)
def mew_denisty(lattice, B, J, U, mew_max, mew_min, z, times):
    mew_arr = np.linspace(mew_min, mew_max, times)
    density_arr = np.zeros(times, dtype=np.float64)

    for t, mew in enumerate(mew_arr):
        density_arr[t] = metropolis_density(lattice, 10000000, B, J, (z*U+mew)/2)

    return density_arr, mew_arr


In [ ]:
B = get_B(100)

In [ ]:
delta = 1*10**-20

In [ ]:
density_arr100, mew_arr100 = mew_denisty(lattice, B, J, U, mew_eq_thy+delta, mew_eq_thy-delta, z, 200)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(mew_arr100, density_arr100, label='100')
plt.xlabel(r"$\mu$")
plt.ylabel('Density')
plt.legend(facecolor='white', framealpha=1)
plt.show()

In [ ]:
B = get_B(150)

In [ ]:
density_arr150, mew_arr150 = mew_denisty(lattice, B, J, U, mew_eq_thy+delta, mew_eq_thy-delta, z, 200)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(mew_arr150, density_arr150, label='150K')
plt.xlabel(r"$\mu$")
plt.ylabel('Density')
plt.legend(facecolor='white', framealpha=1)
plt.show()

In [ ]:
B = get_B(250)

In [ ]:
density_arr250, mew_arr250 = mew_denisty(lattice, B, J, U, mew_eq_thy+delta, mew_eq_thy-delta, z, 200)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(mew_arr250, density_arr250, label='250K')
plt.xlabel(r"$\mu$")
plt.ylabel('Density')
plt.legend(facecolor='white', framealpha=1)
plt.show()

# Converge

In [ ]:
@njit("f8[:](i1[:,:], i8, f8, f8, f8)", nopython=True, nogil=True)
def metropolis_density_arr(density_arr, times, B, J, h):
    density_arr = density_arr.copy()
    net_density = np.zeros(times-1, dtype=np.float64)

    def get_spin(denisty):
        return 2*(denisty+1)

    for t in range(times-1):
        x = np.random.randint(0, N)
        y = np.random.randint(0, N)
        density_i = density_arr[x, y]
        density_f = 1 - density_i
        spin_i = get_spin(density_i)
        spin_f = -spin_i

        E_i = 0
        E_f = 0
        
        if x > 0:
            E_i += -spin_i * density_arr[x-1, y] * J
            E_f += -spin_f * density_arr[x-1, y] * J
        if x < N-1:
            E_i += -spin_i * density_arr[x+1, y] * J
            E_f += -spin_f * density_arr[x+1, y] * J
        if y > 0:
            E_i += -spin_i * density_arr[x, y-1] * J
            E_f += -spin_f * density_arr[x, y-1] * J
        if y < N-1:
            E_i += -spin_i * density_arr[x, y+1] * J
            E_f += -spin_f * density_arr[x, y+1] * J

        E_i += -spin_i*h
        E_f += -spin_f*h

        dE = E_f - E_i
        if (dE > 0) and (np.random.random() < np.exp(-B * dE)):
            density_arr[x, y] = density_f
        elif dE <= 0:
            density_arr[x, y] = density_f
        
        net_density[t] = density_arr.sum()

    return  net_density

In [ ]:
B = get_B(100)

In [ ]:
density_arrP = metropolis_density_arr(lattice, 10000000, B, J, (z*U+mew_eq_thy)/2)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(10000000-1), density_arrP, label='75% of spins started positive')
plt.xlabel('Iteration')
plt.ylabel('Density')
plt.legend(facecolor='white', framealpha=1)
plt.show()

# References

Ising model:
https://www.youtube.com/watch?v=K--1hlv9yv0&t=914s
https://github.com/lukepolson/youtube_channel/blob/main/Python%20Metaphysics%20Series/vid14.ipynb

Lattice gas model:
https://www.pas.rochester.edu/~stte/phy418S21/units/unit_4-6.pdf